<a href="https://colab.research.google.com/github/sda96/repo/blob/master/%EC%A0%84%EC%B2%B4%EC%A0%9C%EC%B6%9C%EC%97%B0%EC%8A%B5%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이콘 스미싱 문자 분류 공모전
###1.사전준비  
- colab에 경로를 설정하여 Mecab 설치  
- Mecab 을 하기위해선 m4 > autoconf > automake 설치필요
- 그외 필요한 패키지 import

###2.Mecab의 사용자사전 설정
- 양식에 맞게 csv파일로 설정해야함  

###3.분석전 전처리및 샘플링
- 전처리는 tokenizer가 아닌 Mecab을 사용하기에 re.sub 사용
- 클래스 0과 1의 비율이 0.5 : 6 으로 비율에 맞게 언더샘플링(임의선택) 시도

###4.단어의 벡터화(Vectorizer,TFIDF) 및 문자의 길이 scaling
- Mecab으로 나누어진 단어들을 vectorizer를 시켯으나 tfidf는 효과가 별로라 생각되엇음
- 0과 1의 클래스의 문장의 길이가 서로 차이가 크게 나기에 MinMaxscale후 벡터화된 단어와 곱하여줌

###5._train과 Y_train 분류 및 class_weight 계산
- 교차검증을 위하여 X와 Y를 나누어 주었고 0과 1의 비율에 맞는 클래스 가중치를 주었음

###6.모델정의및 모델훈련
- 함수정의를 이용하여 모델의 layer를 설정함
- 해당 모델을 LSTM를 사용하였음

###7.예측
- test 데이터에도 train 데이터에 적용한 것을 동일하고 적용후 model.fit(test)


##1.사전준비

In [ ]:
cd /tmp/

/tmp


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 3.8MB 43.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
#경로 이동및 해당 사이트에서 mecab 설치
import os
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  1336k      0  0:00:01  0:00:01 --:--:--  181M


In [ ]:
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz
#압축해제되어 나온 새폴더로 이동
os.chdir('/tmp/mecab-0.996-ko-0.9.2')
#configure는 소스파일에 대한 환경설정을 해주는 명령어다.(configure뒤에 옵션을 넣음) 서버환경에 맞쳐 makefile을 생성해주는 과정이다.
#소스를 사용할 컴퓨터와 사용자의 환경에 맞게 내 컴퓨터는 어떤 기종이고 컴파일에 필요한 시스템 파일들은 어디에 위치해 있으며, 어떤곳에 설치를 하겠다고 지정을 하는 것이다.
# ./configure --prefix = /usr/local/mysql 하게 되면 어떤 파일을 /usr/local/mysql 이라는 곳에 설치 하겠다는 뜻.
!./configure
#make는 소스를 컴파일 하는 것이다. 컴파일이란 것은 소스파일을 사용자가 실행 가능한 파일(_exe)로 만들어 주는 과정을 말한다.
#make 과정이 끝나고 나면 설치파일(Setup 파일 같은 것)이 생성된 상태라고 볼 수 있다.
!make
#필요한 경로나 파일을 확인하고 없으면 생성한다.
!make check
#make install은 make를 통해 만들어진 설치파일(setup)을 설치를 하는 과정이다. 한마디로 build된 프로그램을 실행 할 수 있게 파일들을 알맞은 위치에다가 복사를 한다.
!make install

mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/example.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.c
mecab-0.996-ko-0.9.2/example/example.c
mecab-0.996-ko-0.9.2/example/thread_test.cpp
mecab-0.996-ko-0.9.2/mecab-config.in
mecab-0.996-ko-0.9.2/man/
mecab-0.996-ko-0.9.2/man/Makefile.am
mecab-0.996-ko-0.9.2/man/mecab.1
mecab-0.996-ko-0.9.2/man/Makefile.in
mecab-0.996-ko-0.9.2/mecab.iss.in
mecab-0.996-ko-0.9.2/config.guess
mecab-0.996-ko-0.9.2/README
mecab-0.996-ko-0.9.2/COPYING
mecab-0.996-ko-0.9.2/CHANGES.md
mecab-0.996-ko-0.9.2/README.md
mecab-0.996-ko-0.9.2/INSTALL
mecab-0.996-ko-0.9.2/config.sub
mecab-0.996-ko-0.9.2/configure.in
mecab-0.996-ko-0.9.2/swig/
mecab-0.996-ko-0.9.2/swig/Makefile
mecab-0.996-ko-0.9.2/swig/version.h.in
mecab-0.996-ko-0.9.2/swig/version.h
mecab-0.996-ko-0.9.2/swig/MeCab.i
mecab-0.996-ko-0.9.2/aclocal.m4
mecab-0.996-ko-0.9.2/LGPL
mecab-0.996-ko-0.9.2/Makefile.am
mecab-0.996-

In [ ]:
ls

m4-1.4.18/        mecab-0.996-ko-0.9.2/
m4-1.4.18.tar.gz  mecab-0.996-ko-0.9.2.tar.gz


In [ ]:
# m4 > autoconf > automake 순으로 설치하며 이것을 설치하는 이유는 mecab-ko-dic의 automake의 버전이 낮다는 오류가 생기기 때문이다.
os.chdir('/tmp/')
!curl -LO https://ftp.gnu.org/gnu/m4/m4-1.4.18.tar.gz
!tar xvfz m4-1.4.18.tar.gz
os.chdir('/tmp/m4-1.4.18')
!./configure
!make
!make install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1959k  100 1959k    0     0  1989k      0 --:--:-- --:--:-- --:--:-- 1987k
m4-1.4.18/
m4-1.4.18/ChangeLog-2014
m4-1.4.18/maint.mk
m4-1.4.18/ChangeLog
m4-1.4.18/aclocal.m4
m4-1.4.18/BACKLOG
m4-1.4.18/tests/
m4-1.4.18/tests/getcwd-lgpl.c
m4-1.4.18/tests/test-dup2.c
m4-1.4.18/tests/test-dup-safer.c
m4-1.4.18/tests/test-signbit.c
m4-1.4.18/tests/test-canonicalize-lgpl.c
m4-1.4.18/tests/test-memchr2.c
m4-1.4.18/tests/test-sys_time-c++.cc
m4-1.4.18/tests/opendir.c
m4-1.4.18/tests/test-stdlib.c
m4-1.4.18/tests/test-open.h
m4-1.4.18/tests/test-langinfo-c++.cc
m4-1.4.18/tests/test-mbrtowc-w32-3.sh
m4-1.4.18/tests/test-signal-h-c++2.cc
m4-1.4.18/tests/test-xvasprintf.c
m4-1.4.18/tests/test-getprogname.c
m4-1.4.18/tests/test-dirent.c
m4-1.4.18/tests/test-posix_spawn1.in.sh
m4-1.4.18/tests/test-btowc1.sh
m4-1.4.18/tests/test-stdio.c
m4-1.

In [ ]:
os.chdir('/tmp/')
!curl -LO https://ftp.gnu.org/gnu/autoconf/autoconf-2.69.tar.gz
!tar xvfz autoconf-2.69.tar.gz
os.chdir('/tmp/autoconf-2.69')
!./configure
!make
!make install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1882k  100 1882k    0     0  1889k      0 --:--:-- --:--:-- --:--:-- 1887k
autoconf-2.69/
autoconf-2.69/THANKS
autoconf-2.69/NEWS
autoconf-2.69/bin/
autoconf-2.69/bin/ifnames.in
autoconf-2.69/bin/Makefile.am
autoconf-2.69/bin/autoheader.in
autoconf-2.69/bin/Makefile.in
autoconf-2.69/bin/autoreconf.in
autoconf-2.69/bin/autom4te.in
autoconf-2.69/bin/autoconf.as
autoconf-2.69/bin/autoscan.in
autoconf-2.69/bin/autoupdate.in
autoconf-2.69/man/
autoconf-2.69/man/autoheader.1
autoconf-2.69/man/autoupdate.x
autoconf-2.69/man/Makefile.am
autoconf-2.69/man/autom4te.1
autoconf-2.69/man/ifnames.x
autoconf-2.69/man/Makefile.in
autoconf-2.69/man/autoheader.x
autoconf-2.69/man/autom4te.x
autoconf-2.69/man/autoreconf.x
autoconf-2.69/man/autoscan.x
autoconf-2.69/man/config.sub.x
autoconf-2.69/man/config.sub.1
autoconf-2.69/man/common.x
autocon

In [ ]:
os.chdir('/tmp/')
!curl -LO https://ftp.gnu.org/gnu/automake/automake-1.16.tar.gz
!tar xvfz automake-1.16.tar.gz
os.chdir('/tmp/automake-1.16')
!./configure
!make
!make install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2245k  100 2245k    0     0  2238k      0  0:00:01  0:00:01 --:--:-- 2238k
automake-1.16/
automake-1.16/PLANS/
automake-1.16/PLANS/README
automake-1.16/PLANS/texi/
automake-1.16/PLANS/texi/warnings-for-automake-ng-compatibility.txt
automake-1.16/PLANS/texi/drop-split-info-files.txt
automake-1.16/PLANS/rm-f-without-args.txt
automake-1.16/PLANS/obsolete-removed/
automake-1.16/PLANS/obsolete-removed/am-prog-mkdir-p.txt
automake-1.16/PLANS/obsolete-removed/configure.in.txt
automake-1.16/PLANS/subdir-objects.txt
automake-1.16/README
automake-1.16/doc/
automake-1.16/doc/automake.info
automake-1.16/doc/automake-history.texi
automake-1.16/doc/automake.info-2
automake-1.16/doc/help2man
automake-1.16/doc/automake-history.info
automake-1.16/doc/amhello-1.0.tar.gz
automake-1.16/doc/local.mk
automake-1.16/doc/amhello/
automake-1.16/doc/amh

In [ ]:
ls

AUTHORS         ETN.csv      Makefile.in       Person.csv         VCN.csv
autogen.sh*     feature.def  matrix.def        Place-address.csv  VCP.csv
ChangeLog       Foreign.csv  missing*          Place.csv          VV.csv
char.def        Group.csv    MM.csv            Place-station.csv  VX.csv
clean*          Hanja.csv    model.def         pos-id.def         Wikipedia.csv
CoinedWord.csv  IC.csv       NEWS              Preanalysis.csv    XPN.csv
configure*      Inflect.csv  NNBC.csv          README             XR.csv
configure.ac    INSTALL      NNB.csv           rewrite.def        XSA.csv
COPYING         install-sh*  NNG.csv           right-id.def       XSN.csv
dicrc           J.csv        NNP.csv           Symbol.csv         XSV.csv
EC.csv          left-id.def  NorthKorea.csv    tools/
EF.csv          MAG.csv      NP.csv            unk.def
EP.csv          MAJ.csv      NR.csv            user-dic/
ETM.csv         Makefile.am  Person-actor.csv  VA.csv


In [ ]:
#순서 1,3
import os
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
!tar xvfz mecab-ko-dic-2.1.1-20180720.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.1.1-20180720')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 47.4M  100 47.4M    0     0  21.7M      0  0:00:02  0:00:02 --:--:-- 46.5M
mecab-ko-dic-2.1.1-20180720/
mecab-ko-dic-2.1.1-20180720/configure
mecab-ko-dic-2.1.1-20180720/COPYING
mecab-ko-dic-2.1.1-20180720/autogen.sh
mecab-ko-dic-2.1.1-20180720/Place-station.csv
mecab-ko-dic-2.1.1-20180720/NNG.csv
mecab-ko-dic-2.1.1-20180720/README
mecab-ko-dic-2.1.1-20180720/EF.csv
mecab-ko-dic-2.1.1-20180720/MAG.csv
mecab-ko-dic-2.1.1-20180720/Preanalysis.csv
mecab-ko-dic-2.1.1-20180720/NNB.csv
mecab-ko-dic-2.1.1-20180720/Person-actor.csv
mecab-ko-dic-2.1.1-20180720/VV.csv
mecab-ko-dic-2.1.1-20180720/Makefile.in
mecab-ko-dic-2.1.1-20180720/matrix.def
mecab-ko-dic-2.1.1-20180720/EC.csv
mecab-ko-dic-2.1.1-20180720/NNBC.csv
mecab-ko-dic-2.1.1-20180720/clean
mecab-ko-

In [ ]:
#순서 2
#mecab-ko-dic-2.1.1-20180720를 설치후 make를 돌렸는데 libmecab.so.2: cannot open shared object file: No such file or directory 가 발생하면 실행하는 셀
#이 셀을 실행후 다시 make를 돌리면 된다.
!sudo ldconfig

/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [ ]:
ls

aclocal.m4       EF.csv       Makefile        NR.csv             user-dic/
AUTHORS          EP.csv       Makefile.am     Person-actor.csv   VA.csv
autogen.sh*      ETM.csv      Makefile.in     Person.csv         VCN.csv
autom4te.cache/  ETN.csv      matrix.bin      Place-address.csv  VCP.csv
ChangeLog        feature.def  matrix.def      Place.csv          VV.csv
char.bin         Foreign.csv  missing*        Place-station.csv  VX.csv
char.def         Group.csv    MM.csv          pos-id.def         Wikipedia.csv
clean*           Hanja.csv    model.bin       Preanalysis.csv    XPN.csv
CoinedWord.csv   IC.csv       model.def       README             XR.csv
config.log       Inflect.csv  NEWS            rewrite.def        XSA.csv
config.status*   INSTALL      NNBC.csv        right-id.def       XSN.csv
configure*       install-sh*  NNB.csv         Symbol.csv         XSV.csv
configure.ac     J.csv        NNG.csv         sys.dic
COPYING          left-id.def  NNP.csv         tools/
dicrc        

In [ ]:
#순서 1,3

os.chdir('/tmp/mecab-ko-dic-2.1.1-20180720')
!./autogen.sh
!./configure
!make
# !sh -c 'echo "dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic" > /usr/local/etc/mecabrc'
!make install

Looking in current directory for macros.
configure.ac:2: warning: AM_INIT_AUTOMAKE: two- and three-arguments forms are deprecated.  For more info, see:
configure.ac:2: https://www.gnu.org/software/automake/manual/automake.html#Modernize-AM_005fINIT_005fAUTOMAKE-invocation
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
/tmp/mecab-ko-dic-2.1.1-20180720/missing: Unknown `--is-lightweight' option
Try `/tmp/mecab-ko-dic-2.1.1-20180720/missing --help' for more information
configure: WARNING: 'missing' script is too old or missing
checking for a thread-safe mkdir -p... /bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking for mecab-config... /usr/local/bin/mecab-config
checking that generated files are newer than configure... done
configure: creating ./config.status
config.status: creating Makefile
/usr/local/libexe

In [ ]:
#이후 설치완료된 mecab-ko 와 mecab-ko-dic을 파이썬에서 활용하기 위해 현 파이썬 환경에 mecab.py를 설치한다. 
import os
os.chdir('/content')

!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')

!python3 setup.py build
!python3 setup.py install

Cloning into 'mecab-python-0.996'...
remote: Counting objects: 17, done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 3), reused 0 (delta 0)
Unpacking objects: 100% (17/17), done.
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
copying MeCab.py -> build/lib.linux-x86_64-3.6
running build_ext
building '_MeCab' extension
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/include -I/usr/include/python3.6m -c MeCab_wrap.cxx -o build/temp.linux-x86_64-3.6/MeCab_wrap.o
x86_64-linux-gnu-g++ -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.6/MeCab_wrap.o -L/usr/local/lib -lmecab -lstdc

In [ ]:
cd /tmp/

/tmp


In [ ]:
from google.colab import drive
drive.mount("gdrives")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrives


In [ ]:
cd /tmp/gdrives/My\ Drive/자료파일/fildown/

/tmp/gdrives/My Drive/자료파일/fildown


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame #데이터 전처리
import random
from tqdm import tqdm
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.__version__
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

train = pd.read_csv("train.csv")
test = pd.read_csv("public_test.csv")
submission = pd.read_csv("submission_제출양식.csv")

##2.Mecab의 사용자사전 설정

In [ ]:
cd /tmp/gdrives/My\ Drive/자료파일/fildown/

/tmp/gdrives/My Drive/자료파일/fildown


In [ ]:
dic = pd.read_csv("my_dic.csv")

In [ ]:
cd /tmp/mecab-ko-dic-2.1.1-20180720/user-dic/

/tmp/mecab-ko-dic-2.1.1-20180720/user-dic


In [ ]:
dic.to_csv("dic.csv",index=False)

In [ ]:
ls

dic.csv  nnp.csv  person.csv  place.csv  README.md


In [ ]:
cd /tmp/mecab-ko-dic-2.1.1-20180720/

/tmp/mecab-ko-dic-2.1.1-20180720


In [ ]:
# 새로 넣은 dic.csv를 사전에 추가
! ./tools/add-userdic.sh

In [ ]:
# 새로운 csv 파일을 사전에 적용및 설치
! make clean
! make install

In [ ]:
cd /tmp/gdrives/My\ Drive/자료파일/fildown/

/tmp/gdrives/My Drive/자료파일/fildown


##3.분석전 전처리및 샘플링

In [ ]:
import re
def cleanText(readData):
    result = re.sub('[^ ㄱ-ㅣㅣ가-힣|a-z|A-Z]+',' ',readData)
    result = re.sub('[X]',' ',result)
    result = re.sub('를',' ',result)
    result = re.sub('을',' ',result)
    result = re.sub('는',' ',result)
    result = re.sub('원',' ',result)
    result = re.sub('만',' ',result)
    result = re.sub('분',' ',result)
    result = re.sub('o',' ',result)
    result = re.sub('수신 거부','수신거부',result)
    result = re.sub('으싸',' ',result)
    result = re.sub('ㅂ',' ',result)
    result = re.sub('드립니다',' ',result)
    result = re.sub('그리고',' ',result)
    result = re.sub('지만',' ',result)
    result = re.sub('하지만',' ',result)
    result = re.sub('을',' ',result)
    result = re.sub('다면',' ',result)
    result = re.sub('바랍니다',' ',result)
    result = re.sub('됩니다',' ',result)
    result = re.sub('거나',' ',result)
    result = re.sub('경우',' ',result)
    result = re.sub('까지',' ',result)
    result = re.sub('습니다',' ',result)
    result = re.sub('입니다',' ',result)
    result = re.sub('합니다',' ',result)
    result = re.sub('에서',' ',result)
    result = re.sub('으로',' ',result)
    result = re.sub('셔서',' ',result)
    result = re.sub('으시',' ',result)
    result = re.sub('도록',' ',result)
    return result

In [ ]:
f1 = lambda x : cleanText(x)
clean_train_xx = pd.DataFrame(train.text.map(f1))
clean_test_xx = pd.DataFrame(test.text.map(f1))

In [ ]:
print("clean_train_xx 자료 갯수 : {0} 열이름 : {1}".format(clean_train_xx.shape[0],"text")) 
print("train_yyy 자료 갯수 : {0} 열이름 : {1}".format(train.shape[0],"smishing")) 
print("clean_test_xx 자료 갯수 : {0} 열이름 : {1}".format(clean_test_xx.shape[0],"text")) 
print("test_yyy 자료 갯수 : {0} 열이름 : {1}".format(test.shape[0],"smishing")) 

clean_train_xx 자료 갯수 : 295945 열이름 : text
train_yyy 자료 갯수 : 295945 열이름 : smishing
clean_test_xx 자료 갯수 : 1626 열이름 : text
test_yyy 자료 갯수 : 1626 열이름 : smishing


In [ ]:
lens = pd.DataFrame([len(i) for i in clean_train_xx.text],columns=["lens"])
clean_train_xx_df = pd.concat([clean_train_xx.text,train.smishing,lens],axis=1)
clean_train_xx_df.groupby("smishing").count()

,text,lens
smishing,,
0,277242,277242
1,18703,18703


In [ ]:
clean_train_xx_df = clean_train_xx_df.drop_duplicates(keep="first").reset_index(drop=True)
clean_train_xx_df.groupby("smishing").count()

,text,lens
smishing,,
0,268647,268647
1,14317,14317


In [ ]:
# 0과 1의 비율이 약 99.7%로 데이터 그대로 돌리면 1을 분류하기 어렵기에 비율을 75%로 낮추는 under sampling을 하겠습니다.
# under sampling을 안하고 데이터가 많으면 정확도는 높으나 정밀성이 떨어지기에 정밀성을 높이기 위해서 비율을 조절하였습니다.
# 0과 1의 월별 평균비율은 0.089이므로 이를 기준으로 데이터를 뽑으면 200,000개중 17,819개를 뽑아야 합니다
import random
 #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(clean_train_xx_df[clean_train_xx_df['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 150000 )
train_sm_list=list(clean_train_xx_df[clean_train_xx_df['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 13350 )
under_clean_train_xx_df = clean_train_xx_df.iloc[train_nsmishing+train_smishing,:].reset_index(drop=True)
under_clean_train_xx_df = under_clean_train_xx_df.sample(frac=1).reset_index(drop=True)

In [ ]:
under_clean_train_xx_df.groupby("smishing").count()

,text,lens
smishing,,
0,150000,150000
1,13350,13350


In [ ]:
"""smishing_sms = clean_train_xx_df.loc[clean_train_xx_df.smishing==1,:]
smishing_sms
import konlpy
from konlpy.tag import Mecab
max_words = 1000
# 차원의 수 : 150
max_len = 150
li_train = list(smishing_sms.text)
pos_tagger = Mecab()
posts_tokens = [' '.join(pos_tagger.nouns(sentence)) for sentence in tqdm(li_train)]
vect = CountVectorizer(min_df=1,max_features=max_len).fit(posts_tokens)
count = vect.transform(posts_tokens).toarray().sum(axis=0)
idx = np.argsort(-count)
count = count[idx]
feature_name = np.array(vect.get_feature_names())[idx]
print(list(zip(feature_name, count))[:100])"""

"smishing_sms = clean_train_xx_df.loc[clean_train_xx_df.smishing==1,:]\nsmishing_sms\nimport konlpy\nfrom konlpy.tag import Mecab\nmax_words = 1000\n# 차원의 수 : 150\nmax_len = 150\nli_train = list(smishing_sms.text)\npos_tagger = Mecab()\nposts_tokens = [' '.join(pos_tagger.nouns(sentence)) for sentence in tqdm(li_train)]\nvect = CountVectorizer(min_df=1,max_features=max_len).fit(posts_tokens)\ncount = vect.transform(posts_tokens).toarray().sum(axis=0)\nidx = np.argsort(-count)\ncount = count[idx]\nfeature_name = np.array(vect.get_feature_names())[idx]\nprint(list(zip(feature_name, count))[:100])"

##4.단어의 벡터화(Vectorizer,TFIDF) 및 문자의 길이 scaling

In [ ]:
# batch-size와 같으며 size는 10000개씩 들어간다는 의미이다. (들어가는 단어의 갯수1,000개에서 10,000개를 늘리니 결과가 0.56에서 0.66으로 올랐다.) 1/04
max_words = 10000
# 차원의 수 : (test의 features의 수가 121이기에 121로 고정한다.) 1/04
max_len = 121
li_train = list(under_clean_train_xx_df.text)
# 1번나온 단어는 무시하고 문장별 나온 단어의 인덱스의 갯수(열의 갯수)는 150개이고 단어는 2개씩 묶어서 숫자기반 벡터화 (tfidf보다 더 효과가 좋았따.) 1/04
vectorizer = CountVectorizer(min_df=1,max_features=max_len)
# 1번나온 단어는 무시하고 문장별 나온 단어의 인덱스의 갯수(열의 갯수)는 150개이고 단어는 2개씩 묶어서 문장별 숫자빈도 기반 벡터화   ,ngram_range=(2, 2)
tfidfvectorizer = TfidfVectorizer(min_df=1,max_features=max_len)
import konlpy 
from konlpy.tag import Mecab
pos_tagger = Mecab()
posts_tokens = [' '.join(pos_tagger.nouns(sentence)) for sentence in tqdm(li_train)]

100%|██████████| 163350/163350 [01:07<00:00, 2426.10it/s]


In [ ]:
import konlpy 
from konlpy.tag import Mecab
pos_tagger = Mecab()
pos_tagger.nouns("안녕하세요. 박창현입니다. 현재 충무로영상센터에 있어요")

['안녕', '박창현', '충무로', '영상', '센터']

In [ ]:
vectorizer_posts_tokens = vectorizer.fit_transform(posts_tokens).toarray()
tfidfvectorizer_posts_tokens = tfidfvectorizer.fit_transform(posts_tokens).toarray() 
print(vectorizer_posts_tokens.shape)
print(tfidfvectorizer_posts_tokens.shape)

(163350, 121)
(163350, 121)


In [ ]:
vectorizer.get_feature_names()

In [ ]:
#from google.colab import data_table
#data_table.DataTable(under_clean_train_xx_df, include_index=False, num_rows_per_page=10)

In [ ]:
"""series = pd.read_csv("series.csv",header=0, squeeze=True)
series = series.loc[series.ratio.notnull(),:] 
series = series.iloc[:,[0,4]] 
series = pd.DataFrame(list(series["ratio"]), index=pd.date_range("2017-1-1", periods=24, freq="MS"),columns=["ratio"])
series.plot()
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(series)
plot_pacf(series)
plt.show()
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

diff_1=series.diff(periods=1).iloc[1:]
diff_1.plot()
plot_acf(diff_1)
plot_pacf(diff_1)
plt.show()
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(series, order=(1,0,0))
model_fit = model.fit(trend='nc',full_output=True, disp=1,)
print(model_fit.summary())
plt.plot(series["ratio"], "bo-", label="실제")
plt.plot(model_fit.predict(1), "ro-", label="추정")
plt.legend()
plt.show()
fore = model_fit.forecast(1)
print(fore)"""

'series = pd.read_csv("series.csv",header=0, squeeze=True)\nseries = series.loc[series.ratio.notnull(),:] \nseries = series.iloc[:,[0,4]] \nseries = pd.DataFrame(list(series["ratio"]), index=pd.date_range("2017-1-1", periods=24, freq="MS"),columns=["ratio"])\nseries.plot()\nimport matplotlib.pyplot as plt\nfrom statsmodels.graphics.tsaplots import plot_acf, plot_pacf\n\nplot_acf(series)\nplot_pacf(series)\nplt.show()\nimport matplotlib.pyplot as plt\nimport pandas as pd\nfrom statsmodels.graphics.tsaplots import plot_acf, plot_pacf\n\ndiff_1=series.diff(periods=1).iloc[1:]\ndiff_1.plot()\nplot_acf(diff_1)\nplot_pacf(diff_1)\nplt.show()\nfrom statsmodels.tsa.arima_model import ARIMA\n\nmodel = ARIMA(series, order=(1,0,0))\nmodel_fit = model.fit(trend=\'nc\',full_output=True, disp=1,)\nprint(model_fit.summary())\nplt.plot(series["ratio"], "bo-", label="실제")\nplt.plot(model_fit.predict(1), "ro-", label="추정")\nplt.legend()\nplt.show()\nfore = model_fit.forecast(1)\nprint(fore)'

In [ ]:
"""df1 = pd.DataFrame(bb.get_feature_names(),columns=["feature_name"])
df1["smishing"] = 0
df2 = pd.DataFrame(aa.get_feature_names(),columns=["feature_name"])
df2["smishing"] = 1
df = df1.append(df2)"""

'df1 = pd.DataFrame(bb.get_feature_names(),columns=["feature_name"])\ndf1["smishing"] = 0\ndf2 = pd.DataFrame(aa.get_feature_names(),columns=["feature_name"])\ndf2["smishing"] = 1\ndf = df1.append(df2)'

In [ ]:
#aa = clean_train_xx_df.loc[clean_train_xx_df.smishing==1,:]
#aa = aa.text
#aa = [" ".join(pos_tagger.nouns(sentence)) for sentence in tqdm(aa)]

100%|██████████| 14317/14317 [00:24<00:00, 583.39it/s]


In [ ]:
#pd.Series(str(aa).split()).value_counts()[0:20]

In [ ]:
#data_table.DataTable(pd.DataFrame(aa[:100]), include_index=False, num_rows_per_page=10)

In [ ]:
"""a = list(train[train['smishing']!=1].index)
aa = random.sample(a, 100000 )
aa = train.iloc[aa,:].reset_index(drop=True)
aa = aa.text
aaa = [' '.join(pos_tagger.morphs(sentence)) for sentence in tqdm(aa)]
len_aaa = [len(i) for i in aaa]
pd.DataFrame(len_aaa).describe()"""

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
aa = np.array(under_clean_train_xx_df.lens)
aa = aa.reshape(-1,1)
lens_sc = scaler.fit_transform(aa)

In [ ]:
data = []
for i in range(0,len(vectorizer_posts_tokens)):
  if lens_sc[i] <= 0.11:
    data.append(vectorizer_posts_tokens[i])
  else:
    data.append(vectorizer_posts_tokens[i] * (1+lens_sc[i]) )
data = np.array(data)

In [ ]:
"""from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
noscaler = StandardScaler()
data_tfidf_train_no = noscaler.fit_transform(vectorizer_posts_tokens)
data_tfidf_train_sc = scaler.fit_transform(vectorizer_posts_tokens)"""

##5._train과 Y_train 분류 및 class_weight 계산


In [ ]:
# tfidf보다 vectorize를 하였더니 더 그럴듯하게 나온듯 하다.
#X = data_tfidf_train_no
X1 = data
Y = under_clean_train_xx_df.smishing
X_train,X_test,Y_train,Y_test = train_test_split(X1,Y,test_size=0.15)

In [ ]:
"""from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X_train,Y_train)
pre = MNB.predict_proba(test_data)
submission['smishing'] = pd.DataFrame(pre)
submission.to_csv("MNB5.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장"""

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(under_clean_train_xx_df.smishing),under_clean_train_xx_df.smishing)
class_weights = dict(enumerate(class_weights))

##6.모델정의및 모델훈련

In [ ]:
#신 버전
def LSTM():
    inputs = keras.Input(name='inputs',shape=[max_len])
    # max_words는 들어오는 단어의 집합으로 단어의 갯수라고 볼수있고 10000
    # input_length는 앞으로 들어올 sequence(array)의 길이이고 121
    # output_dim는 말 그대로 embedding후 나온 결과의 길이입니다. 50
    layer = keras.layers.Embedding(max_words,50,input_length=max_len)(inputs)
    layer = keras.layers.LSTM(64)(layer)
    layer = keras.layers.Dense(256,name='FC1')(layer)
    layer = keras.layers.Activation('relu')(layer)
    layer = keras.layers.Dropout(0.5)(layer)
    layer = keras.layers.Dense(1,name='out_layer')(layer)
    layer = keras.layers.Activation('sigmoid')(layer)
    model = keras.Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
# 0.73
# 데이터 많이
# 가중치 추가
model = LSTM()
model.compile(loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=64,epochs=3,class_weight=class_weights,
          validation_split=0.15,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

##7.예측

In [ ]:
test_lens = pd.DataFrame([len(i) for i in clean_test_xx.text],columns=["lens"])
bb = np.array(test_lens.lens)
bb = bb.reshape(-1,1)
test_lens_sc = scaler.fit_transform(bb)


test_li_train = clean_test_xx.text
test_posts_tokens = [' '.join(pos_tagger.morphs(sentence)) for sentence in tqdm(test_li_train)]

test_vectorizer_posts_tokens = vectorizer.fit_transform(test_posts_tokens).toarray()
test_lens_sc = scaler.fit_transform(bb)

test_data = []
for i in range(0,len(test_vectorizer_posts_tokens)):
  if test_lens_sc[i] <= 0.11:
    test_data.append(test_vectorizer_posts_tokens[i])
  else:
    test_data.append(test_vectorizer_posts_tokens[i] * (1+test_lens_sc[i]) )
test_data = np.array(test_data)
predict = model.predict(test_data)
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("LSTM15.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

In [ ]:
"""from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import pandas as pd
skf = KFold(n_splits=4, shuffle=True)

accuracy = []

for train, validation in skf.split(X1, Y):
  model = LSTM()
  model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['accuracy'])
  # 학습 데이터를 이용해서 학습
  model.fit(X1[train],Y[train],batch_size=64,epochs=3,class_weight=class_weights,
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])

  # 테스트 데이터를 이용해서 검증
  k_accuracy = '%.4f' % (model.evaluate(X1[validation], Y[validation])[1])
  accuracy.append(k_accuracy)

# 전체 검증 결과 출력
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
predict = model.predict(test_data)
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("kfold-LSTM6.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장"""

In [ ]:
"""def simpleRNN():
  inputs = keras.Input(name='inputs',shape=[max_len])
  layer = keras.layers.Embedding(max_words,50,input_length=max_len)(inputs)
  layer = keras.layers.SimpleRNN(64)(layer)
  layer = keras.layers.Dense(256,name='FC1')(layer)
  layer = keras.layers.Activation('relu')(layer)
  layer = keras.layers.Dropout(0.8)(layer)
  layer = keras.layers.Dense(1,name='out_layer')(layer)
  layer = keras.layers.Activation('sigmoid')(layer)
  model = keras.Model(inputs=inputs,outputs=layer)
  return model"""

In [ ]:
"""model = simpleRNN()
model.compile(loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=64,epochs=3,class_weight=class_weights,
          validation_split=0.15,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.00001)])
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
predict = model.predict(test_vectorizer_posts_tokens)
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("simpleRNN.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장"""

In [ ]:
"""from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
clf = MultinomialNB()
clf.fit(X_train,Y_train,sample_weight=sample_weight)
pred = clf.predict(X_test)
print(accuracy_score(Y_test,pred))
from sklearn.metrics import classification_report
print(classification_report(Y_test, pred, target_names=['class 0', 'class 1']))
predict = clf.predict_proba(test_vectorizer_posts_tokens)
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("MultinomialNB.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train,Y_train)
pred = clf.predict(X_test)
print(accuracy_score(Y_test,pred))
from sklearn.metrics import classification_report
print(classification_report(Y_test, pred, target_names=['class 0', 'class 1']))
predict = clf.predict_proba(test_vectorizer_posts_tokens)
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("GaussianNB.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
clf = LogisticRegression()
clf.fit(X_train,Y_train)
pred = clf.predict(X_test)
print(accuracy_score(Y_test,pred))
from sklearn.metrics import classification_report
print(classification_report(Y_test, pred, target_names=['class 0', 'class 1']))
predict = clf.predict_proba(test_vectorizer_posts_tokens)
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("LogisticRegression.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(500,500))
clf.fit(X_train,Y_train)
pred = clf.predict(X_test)
print(accuracy_score(Y_test,pred))
from sklearn.metrics import classification_report
print(classification_report(Y_test, pred, target_names=['class 0', 'class 1']))
predict = clf.predict_proba(test_vectorizer_posts_tokens)
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("MLPClassifier.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,Y_train)
pred = clf.predict(X_test)
print(accuracy_score(Y_test,pred))
from sklearn.metrics import classification_report
print(classification_report(Y_test, pred, target_names=['class 0', 'class 1']))
predict = clf.predict_proba(test_vectorizer_posts_tokens)
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("DecisionTreeClassifier.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

from sklearn.svm import SVC
clf = SVC(gamma=0.1,C=1,kernel='rbf',probability=True,class_weight=class_weights)
clf.fit(X_train,Y_train)
pred = clf.predict(X_test)
print(accuracy_score(Y_test,pred))
from sklearn.metrics import classification_report
print(classification_report(Y_test, pred, target_names=['class 0', 'class 1']))
predict = clf.predict_proba(test_vectorizer_posts_tokens)
submission['smishing'] = pd.DataFrame(predict)
submission.to_csv("SVC.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장"""

In [ ]:
"""model1 = keras.Sequential()
model1.add(keras.layers.Embedding(max_words,50,input_length=max_len))
model1.add(keras.layers.LSTM(64))
model1.add(keras.layers.Dense(256,name='FC1'))
model1.add(keras.layers.Activation('relu'))
model1.add(keras.layers.Dropout(0.5))
model1.add(keras.layers.Dense(1,name='out_layer'))
model1.add(keras.layers.Activation('sigmoid'))
model1.summary()"""

In [ ]:
"""from sklearn.metrics import classification_report

y_true = Y_test
y_pred = Y_test_pred

print(classification_report(y_true, y_pred, target_names=['class 0', 'class 1']))"""

In [ ]:
"""from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(Y_test,model.predict(X_test)[:,0])
plt.plot(fpr, tpr, 'o-', ms=2, label="Logistic Regression")
plt.plot([0, 1], [0, 1], 'k--', label="random guess")
plt.xlabel('위양성률(Fall-Out)')
plt.ylabel('재현률(Recall)')
plt.title('ROC 커브')
plt.show()"""